# CommunityResource creation monitoring

- uses the public API
- uses a file to store the latest CommunityResource we've seen in between cron runs
- notifies on Slack (webhook)

In [ ]:
import requests, os
from datetime import datetime
from pathlib import Path

In [ ]:
current_path = Path(os.getenv('WORKING_DIR')) if os.getenv('WORKING_DIR') else Path().absolute()
current_path

In [ ]:
# load config
import configparser
config = configparser.ConfigParser()
config.read(current_path / 'config.ini')
assert 'secrets' in config.sections()

wh_url = config.get('secrets', 'slack_webhook_url')

**Use a file to store the last community resource we've seen so far.**

In [ ]:
last_run = None
path = Path(current_path / 'last-community-resource-run.txt')
if path.exists():
    with path.open() as f:
        last_run = datetime.fromisoformat(f.read().strip())
last_run

**send a message to Slack API**

In [ ]:
def send_message(msg):
    print(msg)
    r = requests.post(wh_url, json={'text': msg})
    assert r.text == 'ok'

In [ ]:
def notify_cr(cr):
    dataset_link = f"<{cr['dataset']['uri']}|{cr['dataset']['title']}>" if cr['dataset'] else '*None*'
    send_message(f":panda_face: new _CommunityResource_ on {dataset_link}: {cr['url']}")

In [ ]:
cr_url = 'https://www.data.gouv.fr/api/1/datasets/community_resources/?sort=-created_at'
r = requests.get(cr_url)
new_last_found = None
for cr in r.json()['data']:
    cr_date = datetime.fromisoformat(cr['created_at'])
    if last_run and cr_date < last_run:
        break
    notify_cr(cr)

In [ ]:
with path.open('w') as f:
    f.write(datetime.now().isoformat())